------------

In [ ]:
:!cabal install --lib pptable
:!cabal install --lib vector
:!cabal install --lib pretty

Resolving dependencies...
Build profile: -w ghc-9.4.5 -O1
In order, the following will be built (use -v for more details):
 - boxes-0.1.5 (lib) (requires download & build)
 - generic-deriving-1.14.5 (lib) (requires download & build)
 - syb-0.7.2.4 (lib) (requires download & build)
 - pptable-0.3.0.0 (lib) (requires download & build)
Downloaded   boxes-0.1.5
Starting     boxes-0.1.5 (lib)
Downloaded   syb-0.7.2.4
Downloaded   generic-deriving-1.14.5
Starting     generic-deriving-1.14.5 (lib)
Downloaded   pptable-0.3.0.0
Building     generic-deriving-1.14.5 (lib)
Building     boxes-0.1.5 (lib)
Installing   boxes-0.1.5 (lib)
Completed    boxes-0.1.5 (lib)
Starting     syb-0.7.2.4 (lib)
Building     syb-0.7.2.4 (lib)
Installing   syb-0.7.2.4 (lib)
Completed    syb-0.7.2.4 (lib)
Installing   generic-deriving-1.14.5 (lib)
Completed    generic-deriving-1.14.5 (lib)
Starting     pptable-0.3.0.0 (lib)
Building     pptable-0.3.0.0 (lib)
Installing   pptable-0.3.0.0 (lib)
Completed    pptable-0.3

Resolving dependencies...
Up to date

Resolving dependencies...
Up to date

In [1]:
{-# LANGUAGE OverloadedStrings #-}

import Data.Aeson
import qualified Data.ByteString.Lazy as B
import Data.Text.Prettyprint.Doc

-- Define a data type to represent the structure of the JSON
data Location = Location
  { database:: String
  , dbdesc:: String
  , host:: String
  , loc_name :: String
  , loc_class :: String
  , loc_directory :: String
  , loc_remote_node :: String
  , loc_remote_login :: String
  , loc_remote_pwd :: String
  , loc_remote_port :: Int
  , loc_db_name :: String
  , loc_db_user :: String
  , loc_description :: String
  } deriving (Show)

instance Pretty Location where
  pretty loc =
    vsep
      [ "Location {"
      , indent 2 ("database = " <> pretty (database loc))
      , indent 2 ("dbdesc = " <> pretty (dbdesc loc))
      , indent 2 ("host = " <> pretty (host loc))   
      , indent 2 ("loc_name = " <> pretty (loc_name loc))
      , indent 2 ("loc_class = " <> pretty (loc_class loc))
      , indent 2 ("loc_directory = " <> pretty (loc_directory loc))
      , indent 2 ("loc_remote_node = " <> pretty (loc_remote_node loc))
      , indent 2 ("loc_remote_login = " <> pretty (loc_remote_login loc))
      , indent 2 ("loc_remote_pwd = " <> pretty (loc_remote_pwd loc))
      , indent 2 ("loc_remote_port = " <> pretty (loc_remote_port loc))
      , indent 2 ("loc_db_name = " <> pretty (loc_db_name loc))
      , indent 2 ("loc_db_user = " <> pretty (loc_db_user loc))
      , indent 2 ("loc_description = " <> pretty (loc_description loc))
      , "}"
      ]

-- Define an instance to decode JSON into the Location data type
instance FromJSON Location where
  parseJSON = withObject "Location" $ \v -> Location  
    <$> v .: "database"
    <*> v .: "dbdesc"
    <*> v .: "host"
    <*> v .: "loc_name"
    <*> v .: "loc_class"
    <*> v .: "loc_directory"
    <*> v .: "loc_remote_node"
    <*> v .: "loc_remote_login"
    <*> v .: "loc_remote_pwd"
    <*> v .: "loc_remote_port"
    <*> v .: "loc_db_name"
    <*> v .: "loc_db_user"
    <*> v .: "loc_description"

In [2]:
data Location' = Location'
  { database':: String
  , dbdesc':: String
  , host':: String
  , loc_name' :: String
  , loc_class' :: String
  , loc_directory'::String
  , loc_remote_node' :: String  -- agent used 
  , loc_remote_login' :: String -- user agent 
  , loc_remote_port' :: String     -- agent port
  , loc_db_user' :: String
  , loc_description' :: String
  , db_node_name :: String      -- infra location where data is stored
  , db_instance :: String  -- logical locaton
  } deriving (Show)
  
instance Pretty Location' where
  pretty loc =
    vsep
      [ "Location {"
      , indent 2 ("database         = " <> pretty (database' loc))
      , indent 2 ("dbdesc           = " <> pretty (dbdesc' loc))
      , indent 2 ("host             = " <> pretty (host' loc))   
      , indent 2 ("loc_class        = " <> pretty (loc_class' loc))
      , indent 3 ("loc_name         = " <> pretty (loc_name' loc))
      , indent 3 ("loc_directory    = " <> pretty (loc_directory' loc))     
      , indent 3 ("loc_remote_node  = " <> pretty (loc_remote_node' loc))
      , indent 3 ("loc_remote_login = " <> pretty (loc_remote_login' loc))
      , indent 3 ("loc_remote_port  = " <> pretty (loc_remote_port' loc))
      , indent 3 ("loc_db_user      = " <> pretty (loc_db_user' loc))
      , indent 3 ("loc_description  = " <> pretty (loc_description' loc))
      , indent 3 ("db_node_name     = " <> pretty (db_node_name loc))
      , indent 3 ("db_instance      = " <> pretty (db_instance loc))
      , "}"
      ]  

In [3]:
loc2Location :: Location -> String -> [String] -> Location'
loc2Location x classSuffix [db_user,node_name,db_instance] = 
                              Location' {  database'= database x
                                         , dbdesc'= dbdesc x
                                         , host'=  host x
                                         , loc_name'  = loc_name x 
                                         , loc_class' = loc_class x ++ classSuffix
                                         , loc_directory' = loc_directory x
                                         , loc_remote_node'  = loc_remote_node x
                                         , loc_remote_login' = loc_remote_login x
                                         , loc_remote_port'  = show (loc_remote_port x)
                                         , loc_db_user'      = db_user                  ---- is it agent user or db user?
                                         , loc_description'  = loc_description x
                                         , db_node_name      = node_name
                                         , db_instance       = db_instance
                                         }
                                         

In [4]:
import Data.Aeson
import qualified Data.Vector as V
import Data.List (intercalate)
import Data.Text (Text, unpack)
import Data.List.Split (splitOn)
import qualified Data.ByteString.Lazy.Char8 as LBS

valueFromString :: LBS.ByteString -> Maybe Value
valueFromString  s = decode  s :: Maybe Value

unjust :: Maybe a -> a
unjust (Just a) = a

strfromString :: Value -> String
strfromString (String x) = unpack  x

unArray :: Value -> [Value]
unArray (Array x )  = V.toList x

unString :: [Value] -> [String]
unString  [String x , String y] = [unpack  x, unpack y]
unString  [String x , Array y]  = [unpack  x , listFromArray (Array y) ]

listFromArray    = intercalate ";" . map strfromString . unArray

getKafkaString   = LBS.pack . last . splitOn "="

parseKafkaString = map(unString . unArray) . unArray . unjust . valueFromString 

getval :: String -> [[String]] -> String
getval key records =
    case filter (\[k, _] -> k == key) records of
        [] -> ""  -- Return an empty string when no match is found
        [[_, value]] -> value
        _ -> error "Multiple matches found"  -- Handle this case as needed

getvals :: [String] -> [[String]] -> [String]
getvals keys lst  = map (`getval` lst) keys

parseKafka = getvals ["urls","ssl_key"] . parseKafkaString . getKafkaString

In [5]:
-- Oracle parse

safehead::[String] -> String
safehead [] = ""
safehead (x:xs) =  x

getOraNode::[String] -> String
getOraNode = last .splitOn "@" . safehead . tail . reverse

-- data quality 
oraPatterFx :: [String]->[String]
oraPatterFx x | length x == 1 =[head x ,"@Not Found","NotFound"]
     | otherwise = x


getOracle :: Location  -> [String]
getOracle  =  (\x-> map ($ x) [head, getOraNode,last]). oraPatterFx .splitOn "/" . loc_db_user

In [ ]:
import Text.Read  (readMaybe) 
import Data.Maybe (fromMaybe )
import Data.List.Split (splitOn)
import Data.List (intercalate)

parseRedshift :: Location  ->[String]
parseRedshift l  = getUSer l : (concat . filterCols . fromMaybe []. parseJson .  getJsonStr)  l  
   where
       getJsonStr  = drop 1 . dropWhile (/= '=') . loc_db_name  -- get loc_db_name and return string after first '='
       parseJson s = readMaybe s :: Maybe [[String]]            -- parse to list
       filterCols  = map tail .  filter (\[k, _] -> k `elem` ["db_node", "db_name"])
       getUSer = head . splitOn "/" . loc_db_user 

In [9]:
parseLoc :: Location  -> Location'
parseLoc l
    --| loc_class l == "redshift"   = loc2Location l ""  $ parseRedshift l
    | loc_class l == "file" && ((head . loc_directory)   l  == '/'    )  = 
                            loc2Location l "_locdir"  [loc_remote_login  l ,loc_remote_node l, loc_directory l]
                            
    | loc_class l == "file" && ((take 4 . loc_directory) l  == "s3s:" )  = 
                            loc2Location l "_s3s" ["###",getBacket l ,getPrefix l]
                            
    | loc_class l == "file" && ((take 5 . loc_directory) l  == "sftp:")  = 
                            loc2Location l "_sftp" ["###",getBacket l ,getsftpFolder l]
                            
    | loc_class l `elem` ["mysql","greenplum","postgresql"]              = 
                            loc2Location l ""   [ getUserMysql l , getNodeMysql l, getInstMysql l]
                            
    | loc_class l == "sqlserver"    = loc2Location l ""   [getUserMsSql l, getNodeMsSql l,getInstMsSql l ]   
    | loc_class l == "teradata"     = loc2Location l ""   [getUserTd l, loc_db_name l,loc_db_name l ] 
    | loc_class l == "salesforce"   = loc2Location l ""   [getUserSf l, getNodeSf l,getInstSf l ] 
    --| loc_class l == "kafka"        = loc2Location l ""   (getKafka  l)
    --| loc_class l == "oracle"       = loc2Location l ""   (getOracle l)
    | otherwise   = loc2Location l ""  ["###","###","###"]   
    where 
            getBacket = head . splitOn "/" . last. splitOn "@" .loc_directory
            getPrefix =  intercalate  "/" .tail . splitOn "/" . last. splitOn "@" .loc_directory
            getsftpNode   = getBacket
            getsftpFolder =  getPrefix
            --
            getNodeMysql  =  head . splitOn "~" . loc_db_name
            getInstMysql  =  last . splitOn "~" . loc_db_name
            getUserMysql  =  head . splitOn "/" . loc_db_user
            --
            getNodeMsSql  =  head . splitOn "\\" . loc_db_name
            getInstMsSql  =  last . splitOn "\\" . loc_db_name
            getUserMsSql  =  getUserMysql
            --
            getUserTd     =  getUserMysql
            --
            getNodeSf     = last .splitOn "//" . loc_directory
            getInstSf     = last .splitOn "@" . head . splitOn "/" . loc_db_user
            getUserSf     = head .splitOn "@" . head . splitOn "/" . loc_db_user
            --
            --getKafka      = getvals ["ssl_key","urls","dummy"] . parseKafkaString . getKafkaString. loc_db_name
    

let kafkaStr = "JSON=[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],[\"ssl_cert\",\"/data/kafka_devcerts/kafka/signed-certificate-from-acm\"],[\"ssl_key\",\"/data/kafka_devcerts/kafka/key.pem\"],[\"ssl_key_pwd\",\"!{Itbo2aix.D3x7/Yp}!\"]]"


parseKafka kafkaStr

------------

(last . splitOn "=" . loc_db_name )  $ lll!!1
(last . splitOn "=" . loc_db_name )  $ ll!!1

## Read Data

In [7]:
import Data.Either


-- Read JSON data from the file
jsonData <- B.readFile "data2.json"
  
-- Parse JSON using eitherDecode function
let parsedData = eitherDecode jsonData :: Either String [Location]

l = fromRight [] parsedData
length l

2596

In [6]:
l!!2

Location {database = "ushvr12", dbdesc = "dev", host = "odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com", loc_name = "sedhp", loc_class = "oracle", loc_directory = "/u01/app/oracle/product/19.3.0.0/dbee_1", loc_remote_node = "ora-mke2-scanp.am.health.ge.com", loc_remote_login = "gehc_hvr5", loc_remote_pwd = "", loc_remote_port = 4344, loc_db_name = "", loc_db_user = "PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes", loc_description = "scdhp connection"}

## Parse Data

In [8]:
l' = map  parseLoc l
length l'

2596

-- helper function to get 
ll =  filter (\x ->  loc_class x  == "oracle" ) l
lll =  filter (\x ->  loc_class' x  == "oracle" ) l'
map pretty   (take 40 lll)

In [148]:
-- oracle

import Data.List (sort)
import Data.List.Split (splitOn)

--x = (sort. nub . map loc_db_user)  (take 10 ll)
x = (sort. nub . map loc_db_user)  (take 10 ll)
data Table  =  Table String String String deriving ( Data, G.Generic, Show)
instance T.Tabulate Table T.ExpandWhenNested

toTable::String -> Table
toTable  x  = Table x "" ""



(T.printTable . map toTable ) x

                                                                                                                                      
@localhosttns                                                                                                                         
HVR_RO/!{yYDHop7YQaeAtZzE}!@prd1aiq.c67zpurmgqn7.us-east-1.rds.amazonaws.com:1521/prd1aiq                                             
PLANT_USER_ODP/!{r/2axxMra/8grrk/}!@ora-mke2-scanp.am.health.ge.com:1521/prd1mes                                                      
gehc_hvr/!{If/T4sAYqjaR2iUm}!@cgehedp2-scan.am.health.ge.com:1525/ebs_prdvcp                                                          
gevcp_view/!{WdJy4HgoWvIZAeY8}!@agehedd2-scan.am.health.ge.com:1525/ebs_devvcp                                                        
hchvrodp/!{/KSo/OBrXqGqNGVrrq2XJZFR}!@/                                                                                               
hchvrodp/!{3IB5rzN9kqoHuJNv}!@/##hchvrodp/!{3IB5rzN9kqo

In [162]:
safehead::[String] -> String
safehead [] = ""
safehead (x:xs) =  x

getOraNode::[String] -> String
getOraNode = last .splitOn "@" . safehead . tail . reverse

-- data quality 
oraPatterFx :: [String]->[String]
oraPatterFx x | length x == 1 =[head x ,"@Not Found","NotFound"]
     | otherwise = x

getOracle :: String -> [String]
getOracle  =  (\x-> map ($ x) [head, getOraNode,last]). oraPatterFx .splitOn "/" --. loc_db_user


In [163]:
data Table  =  Table String String String deriving ( Data, G.Generic, Show)
instance T.Tabulate Table T.ExpandWhenNested

toTable::[String] -> Table
toTable  [x,y,z]  = Table x y z

In [164]:
y = (sort. nub . map loc_db_user) ll
x = map getOracle  y

(T.printTable . map toTable ) x

                                                                                                                           
                              Not Found                                                        NotFound                    
502816858                     agehscq1-c1-scan.am.health.ge.com:1525                           TST17SAL                    
502816858                     agehscq1-c1-scan.am.health.ge.com:1525                           CRP17SAL                    
502816858                     agehscq1-c1-scan.am.health.ge.com:1525                           DEV17SAL                    
502816858                     agehscq1-c1-scan.am.health.ge.com:1525                           CRP17SAL                    
502816858                     agehscq1-c1-scan.am.health.ge.com:1525                           TST17SAL                    
502816858                     agehscq1-c1-scan.am.health.ge.com:1525                           DR27SAL                     
50281685

In [8]:
--pretty $ l!!6
--loc2Location (l!!6) "_RED" ["db_user","node_name","node_instance"]
--loc2Location (l!!6) "_RED"  $ parseRedshift (l!!6)

In [20]:
--l6 =  l!!6
--pretty l6
--l6' = loc2Location l6 "_RED" ["db_user","node_name","node_instance"]
--pretty l6'

 map pretty (filter (\x ->  loc_class' x == "kafka" ) l' )

### Analysis

In [10]:
import Data.List (nub)
x = (nub . map (\x-> map ($ x) [dbdesc,host,database] ))  l

data Table  =  Table String String String deriving ( Data, G.Generic, Show)
instance T.Tabulate Table T.ExpandWhenNested

toTable::[String] -> Table
toTable  [x,y,z]  = Table x y z

(T.printTable . map toTable ) x


: 

In [9]:
import Data.List (nub)

(nub . map  loc_class) l
(nub . map  dbdesc) l
(nub . map  host) l

["redshift","file","oracle","sqlserver","teradata","postgresql","salesforce","mysql","greenplum","kafka"]

["dev","fin-prod","Finance Nprod","US hvr prod","Us innovation"]

["odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com","odp-fin-prod-hvr-metadata.odp.health.ge.com","odp-fin-nprod-hvr-metadata.odp.health.ge.com","odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com"]

In [10]:
 --let l_prd = map pretty (filter (\x ->  (dbdesc' x) `elem` ["fin-prod","US hvr prod"] ) l' )
 let l_prd =   filter (\x ->  (dbdesc' x) `elem` ["fin-prod","US hvr prod"] ) l' 
 length l_prd

1494

 map pretty $ take 5  l_prd

import Text.Pretty.Simple (pPrint)

In [11]:
import Data.List ( sortOn, sortBy)

In [12]:
import Text.PrettyPrint.Tabulate
import qualified GHC.Generics as G
import Data.Data
import qualified Text.PrettyPrint.Tabulate as T
import qualified Data.Map as Map
import qualified Data.List as List
import qualified Data.Vector as Vector

In [13]:
data Table  =  Table String String String deriving ( Data, G.Generic, Show)
instance T.Tabulate Table T.ExpandWhenNested

toTable::[String] -> Table
toTable  [x,y]  = Table x y ""


customCompare :: [String] -> [String] -> Ordering
customCompare xs ys = compare (take 2 xs) (take 2 ys)


In [27]:
import Data.List (nub)
x = (sortBy customCompare  . nub . map (\x-> map ($ x) [loc_remote_node' , loc_remote_port'] ) )   l_prd
(T.printTable . map toTable ) x

                                                                      
                                                            0         
10.229.1.113                                                4343      
10.229.2.236                                                8080      
10.242.109.196                                              4343      
10.242.109.196                                              6080      
10.242.109.196                                              8080      
10.242.109.196                                              8443      
10.242.109.196                                              9995      
10.242.109.235                                              8080      
10.242.109.242                                              18081     
10.242.109.242                                              8443      
10.242.109.242                                              9090      
10.242.109.242                                              9995      
10.242

database'= database x
                                         , dbdesc'= dbdesc x
                                         , host'=  host x
                                         , loc_name'  = loc_name x 
                                         , loc_class' = loc_class x ++ classSuffix
                                         , loc_directory' = loc_directory x
                                         , loc_remote_node'  = loc_remote_node x
                                         , loc_remote_login' = loc_remote_login x
                                         , loc_remote_port'  = loc_remote_port x
                                         , loc_db_user'      = db_user                  ---- is it agent user or db user?
                                         , loc_description'  = loc_description x
                                         , db_node_name      = node_name
                                         , db_instance       = db_instance